<a href="https://colab.research.google.com/github/meahmadkarim/Expression_Classification_/blob/main/Expression_Classification_from_Facial_Images_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/AI Project/data/image

ls: cannot access '/content/drive/MyDrive/AI': No such file or directory
ls: cannot access 'Project/data/image': No such file or directory


In [ ]:
!7za -y x "/content/drive/MyDrive/AI Project/data/image/origin.7z*"


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/AI Project/data/image/                                                       8 files, 8113576419 bytes (7738 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/AI Project/data/image/origin.7z.001
--
Path = /content/drive/MyDrive/AI Project/data/image/origin.7z.001
Type = Split
Physical Size = 1048576000
Volumes = 8
Total Physical Size = 8113576419
----
Path = origin.7z
Size = 8113576419
--
Path = origin.7z
Type = 7z
Physical Size = 8113576419
Headers Size = 863607
Method = LZMA:25
Solid = +
Blocks = 2

  0%      0% - origin/afraid_African_214.jpg            

In [ ]:
!cp -r ./origin "/content/drive/MyDrive/AI Project"

In [ ]:
label_file=r"/content/drive/MyDrive/AI Project/data/label/label.lst"
images_file=r"/content/drive/MyDrive/AI Project/origin"

In [ ]:
from numpy import split
import pandas as pd
data= pd.read_csv(label_file, sep=" ",header=None)
columns="image_name face_id_in_image face_box_top face_box_left face_box_right face_box_bottom face_box_confidence expression_label".split()
data.columns = columns

data.head(10)

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_confidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.93620,0
1,angry_actor_109.jpg,0,31,157,345,219,50.30560,0
2,angry_actor_120.jpg,1,53,53,372,372,13.94340,2
3,angry_actor_13.jpg,0,77,51,362,388,85.81040,3
4,angry_actor_132.jpg,0,95,31,412,476,82.39480,0
5,angry_actor_137.jpg,0,93,468,842,467,88.95190,0
6,angry_actor_139.jpg,0,0,0,1127,1127,33.52480,0
7,angry_actor_14.jpg,0,13,120,200,93,39.75540,0
8,angry_actor_147.jpg,1,119,47,142,214,8.79739,6
9,angry_actor_150.jpg,0,56,263,376,169,81.87920,0


In [ ]:
data.shape

(91793, 8)

In [ ]:
data_cof=data[data.face_box_confidence>50]

In [ ]:
data_cof.shape

(45638, 8)

In [ ]:
import cv2
import os
total_img=[]
total_lab=[]
for i, row in data_cof.sample(10000).iterrows():
  image_name=row["image_name"]
  x = row["face_box_left"]
  y=row["face_box_top"]
  w=row["face_box_right"]
  z=row["face_box_bottom"]
  label=row["expression_label"]
  img_path=os.path.join(images_file,image_name)

  image = cv2.imread(img_path)

  #croping
  if image is not None:
        crop = image [y:z, x:w]
        img_resize = cv2.resize(crop, (64, 64))
        if img_resize is not None:
         total_img.append(img_resize)
         total_lab.append(label)


In [ ]:
import numpy as np
X = np.array(total_img)
Y = np.array(total_lab)

In [ ]:
len(Y)



6660

In [ ]:
len(X)

6660

In [ ]:
x = X / 255.0

In [ ]:
X_reshape = x.reshape((-1,64*64*3))
X_reshape.shape

(6660, 12288)

In [ ]:
import pandas as pd
pd.Series(Y).value_counts()

3    3355
6    1906
5     494
0     289
4     276
1     275
2      65
dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
sampler = SMOTE()
X_smote,Y_smote=sampler.fit_resample(X_reshape, Y)

3    3355
6    3355
1    3355
5    3355
4    3355
0    3355
2    3355
dtype: int64

In [ ]:
x = X_smote.reshape(-1,64,64,3)

In [ ]:
len(Y_smote)

23485

In [ ]:
z = Y_smote

In [ ]:
import numpy as np
from keras.utils import to_categorical
y = to_categorical(z)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_temp, Y_train, Y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

image_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


image_datagen.fit(X_train)


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
num_classes = 7
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(image_datagen.flow(X_train, Y_train, batch_size=32), epochs=50, validation_data=(X_val, Y_val))

Epoch 1/50
514/514 [==============================] - 146s 272ms/step - loss: 1.9069 - accuracy: 0.1989 - val_loss: 1.8408 - val_accuracy: 0.2611
Epoch 2/50
514/514 [==============================] - 130s 253ms/step - loss: 1.8304 - accuracy: 0.2577 - val_loss: 1.7803 - val_accuracy: 0.2682
Epoch 3/50
514/514 [==============================] - 131s 255ms/step - loss: 1.7740 - accuracy: 0.2846 - val_loss: 1.6530 - val_accuracy: 0.3452
Epoch 4/50
514/514 [==============================] - 128s 249ms/step - loss: 1.7218 - accuracy: 0.3183 - val_loss: 1.6049 - val_accuracy: 0.3892
Epoch 5/50
514/514 [==============================] - 133s 259ms/step - loss: 1.6660 - accuracy: 0.3458 - val_loss: 1.5111 - val_accuracy: 0.4209
Epoch 6/50
514/514 [==============================] - 139s 271ms/step - loss: 1.6182 - accuracy: 0.3682 - val_loss: 1.4592 - val_accuracy: 0.4485
Epoch 7/50
514/514 [==============================] - 145s 282ms/step - loss: 1.5675 - accuracy: 0.3959 - val_loss: 1.3870 -

In [ ]:
loss ,accuracy =model.evaluate(X_test,Y_test )
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

111/111 [==============================] - 5s 47ms/step - loss: 0.7395 - accuracy: 0.7224
Test Loss: 0.7395
Test Accuracy: 0.7224
